In [10]:
%%writefile create_db.hql

DROP DATABASE IF EXISTS stackoverflow CASCADE;
CREATE DATABASE stackoverflow LOCATION '/user/jovyan/store/stackoverflow';
USE stackoverflow;

Overwriting create_db.hql


In [46]:
%%writefile external_table.hql

USE stackoverflow;

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    id INT,
    post_type_id TINYINT,
    year INT,
    month INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES
('input.regex'='.*(?=\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)-.*\").*')
LOCATION '/data/stackexchange1000/posts';


Overwriting external_table.hql


In [47]:
%%writefile sample.hql

USE stackoverflow;

set hive.exec.dynamic.partition.mode=nonstrict
set hive.exec.max.dynamic.partitions.pernode=1000;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    id INT,
    post_type_id TINYINT    
) 
PARTITIONED BY (year INT, month INT)
STORED AS SEQUENCEFILE;

FROM posts_sample_external se
INSERT OVERWRITE TABLE posts_sample PARTITION(year, month)
    select se.id, se.post_type_id, year, month;    


Overwriting sample.hql


In [48]:
%%writefile count.hql

USE stackoverflow;

INSERT OVERWRITE LOCAL DIRECTORY 'result' row format delimited fields terminated by '\t' stored as textfile
SELECT year,concat(year, "-", month),count(*) FROM posts_sample group by year,month LIMIT 3;

Overwriting count.hql


In [49]:
!hive -f create_db.hql
!hive -f external_table.hql
!hive -f sample.hql
!hive -f count.hql
!cat result/*|head -1


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 3.275 seconds
OK
Time taken: 0.196 seconds
OK
Time taken: 0.009 seconds

Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.42 seconds
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.045 seconds
OK
Time taken: 0.447 seconds

Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.407 seconds
OK
Time taken: 0.042 seconds
OK
Time taken: 0.399 seconds
Query ID = jovyan_20180108134040_

Partition stackoverflow.posts_sample{year=2013, month=6} stats: [numFiles=1, numRows=477, totalSize=11158, rawDataSize=4770]
Partition stackoverflow.posts_sample{year=2013, month=7} stats: [numFiles=1, numRows=550, totalSize=12857, rawDataSize=5500]
Partition stackoverflow.posts_sample{year=2013, month=8} stats: [numFiles=1, numRows=531, totalSize=12420, rawDataSize=5310]
Partition stackoverflow.posts_sample{year=2013, month=9} stats: [numFiles=1, numRows=511, totalSize=11940, rawDataSize=5110]
Partition stackoverflow.posts_sample{year=2014, month=1} stats: [numFiles=1, numRows=595, totalSize=13892, rawDataSize=5950]
Partition stackoverflow.posts_sample{year=2014, month=10} stats: [numFiles=1, numRows=527, totalSize=12328, rawDataSize=5270]
Partition stackoverflow.posts_sample{year=2014, month=11} stats: [numFiles=1, numRows=501, totalSize=11710, rawDataSize=5010]
Partition stackoverflow.posts_sample{year=2014, month=12} stats: [numFiles=1, numRows=482, totalSize=11273, rawDataSize=482